* Work on multi-way splits here. 
* Write down some of your design considerations here.

* Module is now working for 7-way split. Technically it should be able to run for any number of splits. 

* Write this as function - done.

* Investigate whether there will be an issue with playerturn() module INSIDE the for loop (to be placed in function) being called, as it calls the unmodified player_action() function. That contains a legacy decision = {Sp} piece of code using old variable assignments. Feeling is that nature of the CF in the main function means that it never will get passed a decision = {Sp}, and this is dealt with by player_action_parent and player_action_child. 

* Investigate whether the issue of hastily specified decision = {Sp} and a bust_indicator assignment is redundant. Feeling is that it is - according to CF diagrams, splits are dealt with during the 1st decision-action sequence (whether parent or child); and so by the time you get to PT, either a decision = {S,D} was taken, or player blackjacked, so no need to consider this possibility. 

* Function, for loop and individual iterations now all harmonised -> move on to the score computation.


* Error 1: This code cannot deal with two pairs being drawn after the decision-action sequence in the for loop has determined a split. 

* As the commands inside IF conditional will generate a further split decision and reindex player, which in our case will be itself a pair, a pair and decision = {Sp} will be passed to playerturn(). This will then do nothing' and decision will be recorded in array as 'Sp', bust_indicator as 0, hand will be recorded as pair, which has been UNRESOLVED. So for 2 pairs being drawn, the IF conditional will only call ONE decision-action sequence before moving to resolve the hand, rather than TWO times. 

* Error 2: Following from error 1, this module cannot deal with long runs of the same card (in this example 7), coming after the first decision-action sequence on the iterable, that is indefinite numbers of pairs coming after the 1st decision-action sequence in the for loop iterator has been executed. 

These and some exceptional behaviour involving aces are all dealt with in multiway splits (2).

In [1]:
import pandas as pd
import random

columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
index = [i for i in range(5, 22)]
index2 = ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']
index3 = ['2, 2','3, 3', '4, 4', '5, 5', '6, 6', '7, 7', '8, 8', '9, 9', '10, 10', 'J, J', 'Q, Q', 'K, K', 'A, A']
indices = index + index2 + index3
basic_strategy_table = pd.read_csv('basic-strategy.csv', header=None)
basic_strategy_table.index = indices
basic_strategy_table.columns = columns

basic_strategy_table_no_doubling = pd.read_csv('basic-strategy-no-doubling.csv', header=None)
basic_strategy_table_no_doubling.index = indices
basic_strategy_table_no_doubling.columns = columns

In [2]:
def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

In [3]:
def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

shuffled_shoe = shuffler(deck, total_cards)

def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe. Outputs a message if there is Blackjack
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(shuffled_shoe.pop())
    player.append(shuffled_shoe.pop())
    
    dealer.append(shuffled_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        player_blackjack = 1
        print("Blackjack, house pays out at 3:2")
    else:
        player_blackjack = 0
        print("No Blackjack this time")
        
    return player, dealer, player_blackjack

def decode(action):
    if action == 'Sp':
        message = 'split'
    elif action == 'H':
        message = 'hit'
    elif action == 'D':
        message = 'double down'
    else:
        message = 'stick'
    return message

def basic_player_multiple_hits(player, dealer, player_blackjack):
    
    if len(player) == 2:
        
        if player_blackjack == 1:
            cardvalues = [21]
            decision = 'S'
            print("It's my lucky day")
        else:
            if player[0] == player[1]:
                lookup_pairs = player[0] + ", " + player[1]
                decision = basic_strategy_table.loc[lookup_pairs, dealer[0]]
                print("As I have a pair of {}s, I am going to {}".format(player[0], decode(decision)))
            elif ('A' in player):
                if player[0] == 'A':
                    lookup_soft_total = player[0] + player[1]
                else:
                    lookup_soft_total = player[1] + player[0]
                
                decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft total, I am going to {}".format(decode(decision)))
            else:
                cardvalues = []
                for card in player:
                    cardvalues.append(deck[card][1])
                
                lookup_hard_total = sum(cardvalues)
                decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
                print("I have a total of {}, I am going to {}".format(lookup_hard_total, decode(decision)))
        
    if len(player) > 2: 
        
        if 'A' in player:  # check if the >3 hand has an ace, i.e. is potentially soft.
            if player[0] != 'A':  # re-order dual ace to the left
                ace_index = player.index('A')  
                player[0], player[ace_index] = player[ace_index], player[0]
            
            cardvalues_no_ace = [deck[card][1] for card in player[1::]]
            
            if sum(cardvalues_no_ace) <= 10: # check soft totals
                lookup_soft_total = player[0] + str(sum(cardvalues_no_ace))

                decision = basic_strategy_table_no_doubling.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft {}, that is {} or {}, I am going to {}.".format(11 + sum(cardvalues_no_ace), 1 + sum(cardvalues_no_ace), 11 + sum(cardvalues_no_ace), decode(decision)))
            else:
                lookup_hard_total = deck['A'][1] + sum(cardvalues_no_ace) # variable naming, hard total as ace can only take value 1 now
                
                decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
                print("I have {}, so I am going to {}".format(1 + sum(cardvalues_no_ace), decode(decision)))
        else:
            cardvalues = [deck[card][1] for card in player]
            lookup_hard_total = sum(cardvalues)
            
            decision = basic_strategy_table_no_doubling.loc[lookup_hard_total, dealer[0]]
            print("I have {}, so I am going to {}".format(sum(cardvalues), decode(decision)))
            
    return decision

In [4]:
def player_action(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        multiple_hand_player = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        multiple_hand_player[0].append(drawcard1)
        multiple_hand_player[1].append(drawcard2)
        
        print("My new hands are: " + str(multiple_hand_player))
        
        return multiple_hand_player

In [5]:
def player_turn(player, decision):

    while decision == 'H':
        playerbust_indicator = check_if_bust(player)
        
        if playerbust_indicator == 0:
            print("Continue decision-action sequence")
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            print(decision)
            player = player_action(player, decision)
            print(player)
        else:
            print("Now dealer's turn as I have gone bust, CF takes over")
            break
    else:
        if decision == 'S':
            print("Decision-action sequence terminated by decision = 'S'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = 0
        elif decision == 'D':
            print("Decision-action sequence terminated by decision = 'D'")
            print("Go to dealer, CF takes over")
            playerbust_indicator = check_if_bust(player)
            
            if playerbust_indicator:
                print("Player doubled and busted.")
            else:
                print("Player doubled and did not bust.")
                
        elif decision == 'Sp':  # This part was added hastily - you need to think out the logic of this through.
            playerbust_indicator = check_if_bust(player)
        
    return player, decision, playerbust_indicator

In [6]:
def check_if_bust(player):
    if 'A' in player:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        card_values_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(card_values_no_ace) <= 10:
            bust_indicator = 0
        elif sum(card_values_no_ace) > 20:
            bust_indicator = 1
        else:
            bust_indicator = 0
    else:
        card_values = [deck[card][1] for card in player]
        
        if sum(card_values) > 21:
            bust_indicator = 1
        else:
            bust_indicator = 0
            
    return bust_indicator

In [7]:
# Modify this to have specific 1st split functionality; think about merging later.
# If further pairs are drawn, these need to be appended at the end. Work on this detail later; get overall flow first.
# This is actually an important detail. Get a working, inefficient method done. 

def player_action_parent(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        player_split_hands = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
                    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        player_split_hands[0].append(drawcard1)
        player_split_hands[1].append(drawcard2)
        
        if [hand[0] == hand[1] for hand in player_split_hands] == [True, False]:  # if additional pairs are drawn then these are placed at the end of the list of lists.
            player_split_hands[0], player_split_hands[1] = player_split_hands[1], player_split_hands[0]
        
        print("My new hands are: " + str(player_split_hands))
        
        return player_split_hands

In [18]:
# This subsidiary module, when decision = {Sp}, will need to add a new empty list to the end of player_split_hands.
# It will then need to transfer one card to that last list.
# If another pair, then that needs to be at the end of the list. 
# Will run into namespace issues. Resolve later. 

def player_action_child(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = shuffled_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player  

    if decision == 'Sp':
        
        index = player_split_hands.index(player) 
        
        player_split_hands.insert(index + 1, list(player.pop()))  # Turn one hand with a pair into two separate hands inside player_split_hands
    
        drawcard1 = shuffled_shoe.pop()
        drawcard2 = shuffled_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        player_split_hands[index].append(drawcard1)
        player_split_hands[index + 1].append(drawcard2)
        
        if [hand[0] == hand[1] for hand in player_split_hands[index:index + 2]] == [True, False]: # Ensures that if further pairs are drawn these are not resolved first.
            player_split_hands[index], player_split_hands[index + 1] = player_split_hands[index + 1], player_split_hands[index]
        
        print("My new hands are: {} and {}.".format(player_split_hands[index], player_split_hands[index + 1]))
        
        player = player_split_hands[index] 
        
        return player
    
# Earlier iteration - return None, and only use this function to operate on the list of lists. Then if none, modify loop
# or interrupt a la CF strategy.

# This is where the issue lies. 

In [191]:
def player_multiple_turns(player_split_hands):
    
    decision_list = []
    playerbust_indicator_list = []
    
    for hand in player_split_hands:
        
        decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
        player = player_action_child(hand, decision)
        
        if decision == 'Sp':
            decision = basic_player_multiple_hits(player, dealer, player_blackjack)
            player = player_action_child(player, decision)
            
        player, decision, playerbust_indicator = player_turn(player, decision)
        decision_list.append(decision)
        playerbust_indicator_list.append(playerbust_indicator)
    
    return player_split_hands, decision_list, playerbust_indicator_list

In [199]:
rigged_shoe = []

for card in "2A4A95AA7A27A7277277":
    rigged_shoe.append(card)
    
print(rigged_shoe)

['2', 'A', '4', 'A', '9', '5', 'A', 'A', '7', 'A', '2', '7', 'A', '7', '2', '7', '7', '2', '7', '7']


In [200]:
deck, total_cards = getcards(decks=1)
shuffled_shoe = rigged_shoe

In [201]:
shuffled_shoe = rigged_shoe

In [202]:
dealer

['2']

In [203]:
player, dealer, player_blackjack = openinghand()
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player_split_hands = player_action_parent(player, decision)

No Blackjack this time
As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and 7 in each of my hands
My new hands are: [['7', '7'], ['7', '7']]


In [204]:
player_split_hands, decision_list, playerbust_indicator_list = player_multiple_turns(player_split_hands)

As I have a pair of 7s, I am going to split
After, splitting, I receive 2 and 7 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].
I have a total of 9, I am going to hit
['7', '2', 'A']
Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over
As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and 2 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].
I have a total of 9, I am going to hit
['7', '2', 'A']
Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over
As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and A in each of my hands
My new hands are: ['7', 'A'] and ['7', '7'].
As I have a soft total, I am going 

In [205]:
print(player_split_hands)
print(decision_list)
print(playerbust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5', '9'], ['7', 'A'], ['A', '4', '7']]
['S', 'S', 'S', 'S', 'S', 'S', 'D']
[0, 0, 0, 0, 0, 0, 0]


In [104]:
decision_list = []
player_bust_indicator_list = []

for hand in player_split_hands:
    
    print("Loop is working on hand: {}".format(hand))
    
    decision = basic_player_multiple_hits(hand, dealer, player_blackjack)
    player = player_action_child(hand, decision)
    
    if decision == 'Sp':
        decision = basic_player_multiple_hits(player, dealer, player_blackjack) # Issue with loop was that hand was being passed as argument here
        player = player_action_child(player, decision)

    player, decision, playerbust_indicator = player_turn(player, decision)
    decision_list.append(decision)
    player_bust_indicator_list.append(playerbust_indicator)
    
    print("The list of lists is {}".format(player_split_hands))

Loop is working on hand: ['7', '7']
As I have a pair of 7s, I am going to split
After, splitting, I receive 2 and 7 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].
I have a total of 9, I am going to hit
['7', '2', 'A']
Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over
The list of lists is [['A', '2', '7'], ['7', '7'], ['7', '7']]
Loop is working on hand: ['7', '7']
As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and 2 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].
I have a total of 9, I am going to hit
['7', '2', 'A']
Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over
The list of lists is [['A', '2', '7'], ['A', 

In [ ]:
# The player_turn module needs to be rewritten to allow for when previous decisions are to split!. That's one problem for tomorrow. 

In [105]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5', '9'], ['7', 'A'], ['A', '4', '7']]
['S', 'S', 'S', 'S', 'S', 'S', 'D']
[0, 0, 0, 0, 0, 0, 0]


In [106]:
# If you can get manual iterations to harmonise with the loop; then you have gotten near to correct answer. Harmonised.

In [116]:
deck, total_cards = getcards(decks=1)
shuffled_shoe = shuffler(deck, total_cards)

In [117]:
shuffled_shoe = rigged_shoe

In [118]:
decision_list = []
player_bust_indicator_list = []

In [119]:
player, dealer, player_blackjack = openinghand()
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player_split_hands = player_action_parent(player, decision)

No Blackjack this time
As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and 7 in each of my hands
My new hands are: [['7', '7'], ['7', '7']]


In [134]:
# Iteration 0 for hand = [7,7]

decision = basic_player_multiple_hits(player_split_hands[0], dealer, player_blackjack)
player = player_action_child(player_split_hands[0], decision)

As I have a pair of 7s, I am going to split
After, splitting, I receive 2 and 7 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].


In [135]:
print(player, decision, player_split_hands)

['7', '2'] Sp [['7', '2'], ['7', '7'], ['7', '7']]


In [136]:
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action_child(player, decision)

I have a total of 9, I am going to hit
['7', '2', 'A']


In [137]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [138]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['7', '7'], ['7', '7']]
['S']
[0]


In [139]:
# Iteration 1 for hand = [7,7]

decision = basic_player_multiple_hits(player_split_hands[1], dealer, player_blackjack)
player = player_action_child(player_split_hands[1], decision)

As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and 2 in each of my hands
My new hands are: ['7', '2'] and ['7', '7'].


In [140]:
print(player, decision, player_split_hands)

['7', '2'] Sp [['A', '2', '7'], ['7', '2'], ['7', '7'], ['7', '7']]


In [141]:
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action_child(player, decision)

I have a total of 9, I am going to hit
['7', '2', 'A']


In [142]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Continue decision-action sequence
As I have a soft 20, that is 10 or 20, I am going to stick.
S
['A', '2', '7']
['A', '2', '7']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [143]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', '7'], ['7', '7']]
['S', 'S']
[0, 0]


In [144]:
# Iteration 2 - hand [7,7]

decision = basic_player_multiple_hits(player_split_hands[2], dealer, player_blackjack)
player = player_action_child(player_split_hands[2], decision)

As I have a pair of 7s, I am going to split
After, splitting, I receive 7 and A in each of my hands
My new hands are: ['7', 'A'] and ['7', '7'].


In [145]:
print(player, decision, player_split_hands)

['7', 'A'] Sp [['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', '7'], ['7', '7']]


In [146]:
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action_child(player, decision)

As I have a soft total, I am going to stick
['7', 'A']


In [147]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [148]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', '7'], ['7', '7']]
['S', 'S', 'S']
[0, 0, 0]


In [149]:
# Iteration 3 hand [7,7]

decision = basic_player_multiple_hits(player_split_hands[3], dealer, player_blackjack)
player = player_action_child(player_split_hands[3], decision)

As I have a pair of 7s, I am going to split
After, splitting, I receive A and 5 in each of my hands
My new hands are: ['7', 'A'] and ['7', '5'].


In [150]:
print(player, decision, player_split_hands)

['7', 'A'] Sp [['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5'], ['7', '7']]


In [151]:
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action_child(player, decision)

As I have a soft total, I am going to stick
['7', 'A']


In [152]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [153]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5'], ['7', '7']]
['S', 'S', 'S', 'S']
[0, 0, 0, 0]


In [154]:
# Iteration 4 - hand[7,5]

decision = basic_player_multiple_hits(player_split_hands[4], dealer, player_blackjack)
player = player_action_child(player_split_hands[4], decision)

I have a total of 12, I am going to hit
['7', '5', '9']


In [155]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Continue decision-action sequence
I have 21, so I am going to stick
S
['7', '5', '9']
['7', '5', '9']
Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [156]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5', '9'], ['7', '7']]
['S', 'S', 'S', 'S', 'S']
[0, 0, 0, 0, 0]


In [157]:
# Iteration 5 - hand [7,7]

decision = basic_player_multiple_hits(player_split_hands[5], dealer, player_blackjack)
player = player_action_child(player_split_hands[5], decision)

As I have a pair of 7s, I am going to split
After, splitting, I receive A and 4 in each of my hands
My new hands are: ['7', 'A'] and ['7', '4'].


In [158]:
decision = basic_player_multiple_hits(player, dealer, player_blackjack)
player = player_action_child(player, decision)

As I have a soft total, I am going to stick
['7', 'A']


In [159]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Decision-action sequence terminated by decision = 'S'
Go to dealer, CF takes over


In [160]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5', '9'], ['7', 'A'], ['7', '4']]
['S', 'S', 'S', 'S', 'S', 'S']
[0, 0, 0, 0, 0, 0]


In [161]:
# Iteration 6 - hand [7,4]

decision = basic_player_multiple_hits(player_split_hands[6], dealer, player_blackjack)
player = player_action_child(player_split_hands[6], decision)

I have a total of 11, I am going to double down
['7', '4', 'A']


In [162]:
player, decision, playerbust_indicator = player_turn(player, decision)
decision_list.append(decision)
player_bust_indicator_list.append(playerbust_indicator)

Decision-action sequence terminated by decision = 'D'
Go to dealer, CF takes over
Player doubled and did not bust.


In [163]:
print(player_split_hands)
print(decision_list)
print(player_bust_indicator_list)

[['A', '2', '7'], ['A', '2', '7'], ['7', 'A'], ['7', 'A'], ['7', '5', '9'], ['7', 'A'], ['A', '4', '7']]
['S', 'S', 'S', 'S', 'S', 'S', 'D']
[0, 0, 0, 0, 0, 0, 0]


In [236]:
# The challenge for this in context of CF occurs in context after a split has already occurred. 
# So after the 1st split, say one, or both of the hands will have pairs. 

# As decision-action-turn sequences are called on these new hands (which will yield a split), the player_action_child()
# module will create two (or four) extra hands within the player_split_hands list of lists. The current decision-action-turn 
# sequence then needs to be halted, and decision-action-sequences need to be called on the newly split hands. 

# Current question is whether or not it is possible for the contents of a for loop to modify the iterator over which it is
# iterating so that the iteration length is extended. 

for w in foo:
    foo.append(1)
    if len(foo) > 5:
        break